In [1]:
from line_solver import *
import numpy as np
GlobalConstants.setVerbose(VerboseLevel.STD)

In [2]:

model = Network('model')

node = np.empty(2, dtype=object)
node[0] = Delay(model, 'Delay')
node[1] = Queue(model, 'Queue1', SchedStrategy.PS)
node[1].setNumberOfServers(2)

jobclass = np.empty(3, dtype=object)
jobclass[0] = ClosedClass(model, 'Class1', 2, node[0], 0)
jobclass[1] = ClosedClass(model, 'Class2', 0, node[0], 0)
jobclass[2] = ClosedClass(model, 'Class3', 1, node[0], 0)

node[0].setService(jobclass[0], Erlang(3, 2))
node[0].setService(jobclass[1], HyperExp(0.5, 3.0, 10.0))
node[0].setService(jobclass[2], Exp.fitMean(1))

node[1].setService(jobclass[0], HyperExp(0.1, 1.0, 10.0))
node[1].setService(jobclass[1], Exp.fitMean(2))
node[1].setService(jobclass[2], Exp.fitMean(3))

P = model.initRoutingMatrix()

# P{1,1} = [0.3,0.1; 0.2,0]
P.set(jobclass[0], jobclass[0], node[0], node[0], 0.3)
P.set(jobclass[0], jobclass[0], node[0], node[1], 0.1)
P.set(jobclass[0], jobclass[0], node[1], node[0], 0.2)

# P{1,2} = [0.6,0; 0.8,0]
P.set(jobclass[0], jobclass[1], node[0], node[0], 0.6)
P.set(jobclass[0], jobclass[1], node[1], node[0], 0.8)

# P{2,1} = [0,0; 1,0]
P.set(jobclass[1], jobclass[0], node[1], node[0], 1.0)

# P{2,2} = [0,1; 0,0]
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)

# P{3,3} = circul(M)
P.set(jobclass[2], jobclass[2], node[0], node[1], 1.0)
P.set(jobclass[2], jobclass[2], node[1], node[0], 1.0)

model.link(P)

In [3]:
solver = np.array([], dtype=object)
solver = np.append(solver, SolverCTMC(model))
solver = np.append(solver, SolverJMT(model, 'seed', 23000, 'samples', 5000, 'verbose', True))
solver = np.append(solver, SolverSSA(model, 'seed', 23000, 'samples', 5000, 'verbose', True))
solver = np.append(solver, SolverFluid(model))
solver = np.append(solver, SolverMVA(model))
solver = np.append(solver, SolverNC(model, 'exact'))
solver = np.append(solver, SolverMAM(model))
#solver = np.append(solver, LINE(model, 'seed', 23000))

AvgTable = np.empty(len(solver), dtype=object)
AvgChainTable = np.empty(len(solver), dtype=object)
AvgSysChainTable = np.empty(len(solver), dtype=object)

for s in range(len(solver)):
    print(f'\nSOLVER: {solver[s].getName()}')
    AvgTable[s] = solver[s].getAvgTable()
    AvgChainTable[s] = solver[s].getAvgChainTable()
    #AvgSysChainTable[s] = solver[s].getAvgSysTable() #TODO, fails



SOLVER: SolverCTMC
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay   Class1  0.5295  0.5295  0.6667  0.3968  0.7942  0.7942
1   Delay   Class2  0.1170  0.1170  0.2167  0.0877  0.5401  0.5401
2   Delay   Class3  0.2177  0.2177  1.0000  1.0000  0.2177  0.2177
3  Queue1   Class1  0.0186  0.0075  0.2348  0.0140  0.0794  0.0794
4  Queue1   Class2  1.3349  0.5401  2.4716  1.0004  0.5401  0.5401
5  Queue1   Class3  0.7823  0.3266  3.5931  3.5931  0.2177  0.2177
  Station   Chain    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay  Chain1  0.6465  0.6465  0.4845  0.2717  1.3343  1.3343
1   Delay  Chain2  0.2177  0.2177  1.0000  1.0000  0.2177  0.2177
2  Queue1  Chain1  1.3535  0.5476  2.1848  0.8739  0.6195  0.6195
3  Queue1  Chain2  0.7823  0.3266  3.5931  3.5931  0.2177  0.2177

SOLVER: SolverJMT
JMT Model: /tmp/workspace/jsim/15783358031285846237/jmodel.jsim
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay   Class1  0.5328  0.5